In [1]:
import pandas as pd
import re
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from category_encoders import OneHotEncoder

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [2]:
df = pd.read_csv(os.path.join('..', 'data', 'filter-data-cleaned.csv'))
filter_list = ['F5', 'F6', 'F7', 'F8', 'F9', 'G2', 'G3', 'G4', 'H11', 'H13', 'H14', 'H3', 'M5', 'M6']

In [3]:
df_clean = df.drop(columns=['Depth','Pockets','description','item_type', 'dimensions', 'Date'])
df_clean = df_clean.dropna()
df_clean.shape

(3780, 8)

In [4]:
def relative_error(y_true, y_pred):
    return np.abs(y_true - y_pred) / y_true


def relative_error_loss_function(y_true, y_pred):
    return np.mean(relative_error(y_true, y_pred))


def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)


def scoring_function(clf, X, y):
    y_pred = clf.predict(X)
    return relative_error_loss_function(y, y_pred)


def fit_grid_search(cv, X_train, y_train):
    cv.fit(X_train, y_train)

    return cv.best_estimator_, cv.best_params_, cv.best_score_


def test_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    relative_err = relative_error(y_test, y_pred)

    mean_relative_err = np.mean(relative_err)
    relative_err_std_dev = np.std(relative_err)
    relative_err_median = np.median(relative_err)
    mse = mean_squared_error(y_test, y_pred)

    return mean_relative_err, relative_err_std_dev, relative_err_median, mse

In [5]:
df_model = df_clean.drop(columns=['company_name', 'quote_id'])
df_model.head(3)

,qty,unit_price,filter_efficiency,Length,Height,Gutter
0,36.0,7.5,G4,625.0,500.0,50.0
1,36.0,27.5,F7,625.0,500.0,50.0
2,16.0,28.5,F7,592.0,490.0,48.0


## Train and evaluate the models

In [6]:
X = df_model.drop(columns=['unit_price'])
y = df_model['unit_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

ohe = OneHotEncoder(cols=['filter_efficiency'], use_cat_names=True)
X_train = ohe.fit_transform(X_train)
X_test = ohe.transform(X_test)

In [7]:
too_few_data_points_filters = df_model['filter_efficiency'].value_counts()[df_model['filter_efficiency'].value_counts() < 10].index

df_model = df_model[~df_model['filter_efficiency'].isin(too_few_data_points_filters)]


In [8]:
X = df_model.drop(columns=['unit_price'])
y = df_model['unit_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=X['filter_efficiency'])

ohe = OneHotEncoder(cols=['filter_efficiency'], use_cat_names=True)
X_train = ohe.fit_transform(X_train)
X_test = ohe.transform(X_test)
X_train.head(3)

,qty,filter_efficiency_G4,filter_efficiency_F7,filter_efficiency_F9,filter_efficiency_G3,filter_efficiency_M6,filter_efficiency_M5,filter_efficiency_H13,filter_efficiency_G2,filter_efficiency_H14,filter_efficiency_F8,Length,Height,Gutter
3333,4.0,1,0,0,0,0,0,0,0,0,0,600.0,320.0,30.0
3165,3.0,1,0,0,0,0,0,0,0,0,0,287.0,592.0,45.0
1977,4.0,0,1,0,0,0,0,0,0,0,0,530.0,500.0,98.0


In [9]:
filter_list = [
    df_model['filter_efficiency'].unique(),
    ['G4'],
    ['M5'],
    ['F7'],
]

results = []


def run_regression(model, model_name, param_grid, X_train, y_train, X_test, y_test):
    cv = GridSearchCV(model, param_grid=param_grid, cv=5, scoring=scoring_function)
    best_model, best_params, val_loss = fit_grid_search(cv, X_train, y_train)
    mean_relative_error, std_dev_relative_error, median_relative_error, mse = test_model(best_model, X_test, y_test)
    return {
        'model_type': model_name,
        'model': best_model.__class__.__name__,
        'best_model': best_model,
        'best_params': best_params,
        'mean_relative_error_val': val_loss,
        'mean_relative_error_test': mean_relative_error,
        'std_dev_relative_error_test': std_dev_relative_error,
        'median_relative_error_test': median_relative_error,
        'mse_test': mse,
    }


def create_filter_mask(filters, df):
    mask = np.zeros(df.shape[0], dtype=bool)
    for filt in filters:
        mask = mask | df[f'filter_efficiency_{filt}'] == 1
    return mask


def execute_train(X_train_param, y_train_param, X_test_param, y_test_param):
    results = []

    for filters in filter_list:
        train_mask = create_filter_mask(filters, X_train_param)
        test_mask = create_filter_mask(filters, X_test_param)

        model_name = 'general' if len(filters) > 1 else filters[0]

        print(f'Training {model_name} model...')

        X_train, X_test = X_train_param[train_mask], X_test_param[test_mask]
        y_train, y_test = y_train_param[train_mask], y_test_param[test_mask]

        # Convert to numpy arrays (this will avoid an error on the gradio side)
        X_train, X_test, y_train, y_test = X_train.values, X_test.values, y_train.values, y_test.values

        # Train Linear Regression
        print('\tLinear regression...', end='')
        results.append(
            run_regression(LinearRegression(), model_name, {}, X_train, y_train, X_test, y_test)
        )
        print('done.')

        # Train SVR
        print('\tSVR regression...', end='')
        param_grid = {
            'C': [1, 10, 100, 1000],
            'kernel': ['rbf','poly']
        }
        results.append(
            run_regression(SVR(), model_name, param_grid, X_train, y_train, X_test, y_test)
        )
        print('done.')

        # Train Decision Tree
        print('\tDecision Tree regression...', end='')
        param_grid={
            'max_depth': [10, 100],
            'min_samples_split': [2, 5, 10],
            'max_features': ('sqrt', 'log2')
        }
        results.append(
            run_regression(DecisionTreeRegressor(), model_name, param_grid, X_train, y_train, X_test, y_test)
        )
        print('done.')

        # Train Random Forest
        print('\tRandom Forest regression...', end='')
        param_grid={
            'n_estimators': [10, 100],
            'max_depth': [10, 100],
            'min_samples_split': [2, 5, 10],
            'max_features': ( 'sqrt', 'log2'),
        }
        results.append(
            run_regression(RandomForestRegressor(), model_name, param_grid, X_train, y_train, X_test, y_test)
        )
        print('done.')

        # Train KNN
        print('\tKNeighbours regression...', end='')
        param_grid={
            'n_neighbors': [2, 3, 5, 7, 10, 15, 17, 20],
            'weights': ['uniform', 'distance'],
            'p': [1, 2, 3]
        }
        results.append(
            run_regression(KNeighborsRegressor(), model_name, param_grid, X_train, y_train, X_test, y_test)
        )
        print('done.')
    
    return results

results_df = pd.DataFrame(execute_train(X_train, y_train, X_test, y_test))

Training general model...
	Linear regression...done.
	SVR regression...done.
	Decision Tree regression...done.
	Random Forest regression...done.
	KNeighbours regression...done.
Training G4 model...
	Linear regression...done.
	SVR regression...done.
	Decision Tree regression...done.
	Random Forest regression...done.
	KNeighbours regression...done.
Training M5 model...
	Linear regression...done.
	SVR regression...done.
	Decision Tree regression...done.
	Random Forest regression...done.
	KNeighbours regression...done.
Training F7 model...
	Linear regression...done.
	SVR regression...done.
	Decision Tree regression...done.
	Random Forest regression...done.
	KNeighbours regression...done.


In [10]:
results_df.sort_values(by=['model_type', 'mean_relative_error_val'], ascending=True)

,model_type,model,best_model,best_params,mean_relative_error_val,mean_relative_error_test,std_dev_relative_error_test,median_relative_error_test,mse_test
18,F7,RandomForestRegressor,"(DecisionTreeRegressor(max_depth=100, max_feat...","{'max_depth': 100, 'max_features': 'sqrt', 'mi...",0.179728,0.140177,0.151479,0.092301,34.273179
17,F7,DecisionTreeRegressor,"DecisionTreeRegressor(max_depth=10, max_featur...","{'max_depth': 10, 'max_features': 'sqrt', 'min...",0.208981,0.152245,0.199963,0.090616,47.118593
19,F7,KNeighborsRegressor,KNeighborsRegressor(n_neighbors=20),"{'n_neighbors': 20, 'p': 2, 'weights': 'uniform'}",0.213533,0.204152,0.268032,0.118674,64.085009
16,F7,SVR,"SVR(C=1, kernel='poly')","{'C': 1, 'kernel': 'poly'}",0.321258,0.271851,0.405244,0.168327,142.308662
15,F7,LinearRegression,LinearRegression(),{},0.324030,0.284150,0.318609,0.188804,115.755277
6,G4,SVR,SVR(C=1),"{'C': 1, 'kernel': 'rbf'}",0.136532,0.147483,0.353769,0.099333,23.818224
8,G4,RandomForestRegressor,"(DecisionTreeRegressor(max_depth=10, max_featu...","{'max_depth': 10, 'max_features': 'sqrt', 'min...",0.139236,0.151361,0.338993,0.108095,13.848584
9,G4,KNeighborsRegressor,"KNeighborsRegressor(n_neighbors=2, p=3)","{'n_neighbors': 2, 'p': 3, 'weights': 'uniform'}",0.161223,0.170063,0.380116,0.104790,18.988469
7,G4,DecisionTreeRegressor,"DecisionTreeRegressor(max_depth=10, max_featur...","{'max_depth': 10, 'max_features': 'sqrt', 'min...",0.165043,0.164722,0.323984,0.105593,15.636936
5,G4,LinearRegression,LinearRegression(),{},0.184920,0.192474,0.372250,0.142876,22.184541


## Get the best model for each filter efficiency

In [11]:
best_models = results_df.sort_values(by='mean_relative_error_val')

# Get results for unique column filters
best_models_unique = best_models.drop_duplicates(subset='model_type')
best_models_unique

,model_type,model,best_model,best_params,mean_relative_error_val,mean_relative_error_test,std_dev_relative_error_test,median_relative_error_test,mse_test
6,G4,SVR,SVR(C=1),"{'C': 1, 'kernel': 'rbf'}",0.136532,0.147483,0.353769,0.099333,23.818224
18,F7,RandomForestRegressor,"(DecisionTreeRegressor(max_depth=100, max_feat...","{'max_depth': 100, 'max_features': 'sqrt', 'mi...",0.179728,0.140177,0.151479,0.092301,34.273179
11,M5,SVR,"SVR(C=10, kernel='poly')","{'C': 10, 'kernel': 'poly'}",0.318551,0.276328,0.225413,0.251797,55.552965
3,general,RandomForestRegressor,"(DecisionTreeRegressor(max_depth=10, max_featu...","{'max_depth': 10, 'max_features': 'log2', 'min...",0.360272,0.366220,2.493265,0.142566,60.316050


## Print the model_type, model and params

In [14]:
for i, row in best_models_unique.iterrows():
    # Print model_type, model and params
    print(f'Model type: {row["model_type"]}')
    print(f'Model: {row["model"]}')
    print(f'Best params: {row["best_params"]}')
    
    print()


Model type: G4
Model: SVR
Best params: {'C': 1, 'kernel': 'rbf'}

Model type: F7
Model: RandomForestRegressor
Best params: {'max_depth': 100, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 10}

Model type: M5
Model: SVR
Best params: {'C': 10, 'kernel': 'poly'}

Model type: general
Model: RandomForestRegressor
Best params: {'max_depth': 10, 'max_features': 'log2', 'min_samples_split': 10, 'n_estimators': 10}



## Save the best models

In [12]:
for model_data in best_models_unique.to_dict(orient='records'):
    model_name = model_data['model_type'].lower()

    with open(os.path.join('..', 'models', f'best_model_{model_name}.pkl'), 'wb') as f:
        pickle.dump(model_data['best_model'], f)

In [13]:
features = X_train.columns
features

Index(['qty', 'filter_efficiency_G4', 'filter_efficiency_F7',
       'filter_efficiency_F9', 'filter_efficiency_G3', 'filter_efficiency_M6',
       'filter_efficiency_M5', 'filter_efficiency_H13', 'filter_efficiency_G2',
       'filter_efficiency_H14', 'filter_efficiency_F8', 'Length', 'Height',
       'Gutter'],
      dtype='object')